<a href="https://colab.research.google.com/github/Humberto3280/floramarilla/blob/main/Automatizar_TC1_TC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ARCHIVO DE AUTOMATIZACIÓN DE TARIFAS (TC1 - TC2)**

In [ ]:
!pip install pandas pyxlsb

In [ ]:
import warnings

# **Lectura de los archivos necesarios para el proceso**



* *TC1*
* *TC2*
* *AP*
* *Davipola*

In [ ]:
import pandas as pd

# Lectura en xlsb
#tc1 = pd.read_excel('TC1.xlsb', engine='pyxlsb')
#tc2 = pd.read_excel('TC2.xlsb', engine='pyxlsb')
#archivo_ap = pd.read_excel('ArchivoAP.xlsx')

#Lectura en csv
#tc1 = pd.read_csv('TC1.csv', delimiter=';', encoding='latin1')
#tc2 = pd.read_csv('TC2.csv', delimiter='|', encoding='latin1')
tc1 = pd.read_csv('TC1.csv')
#tc2 = pd.read_csv('TC2.csv')
#archivo_ap = pd.read_csv('ArchivoAP.csv')

#Lectura en xlsx
#tc1 = pd.read_excel('TC1.xlsx')
tc2 = pd.read_excel('TC2.xlsx')
archivo_ap = pd.read_excel('AP.xlsx')
davipola = pd.read_excel('Dane_Divipola_08_2012.xlsx')

<ipython-input-15-64c61fb599fe>:11: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  tc1 = pd.read_csv('TC1.csv')


In [ ]:
# Imprimir nombres de columnas de TC1 y TC2 para verificar
print("Columnas en TC1:", tc1.columns)
print("Columnas en TC2:", tc2.columns)

Columnas en TC1: Index(['NIU', 'CODIGO DE CONEXION', 'TIPO DE CONEXION', 'NIVEL DE TENSION',
       'NIVEL DE TENSION PRIMARIO', 'PORCENTAJE PROPIEDAD DEL ACTIVO',
       'CONEXION DE RED', 'ID COMERCIALIZADOR', 'ID DEL MERCADO',
       'GRUPO DE CALIDAD', 'CODIGO FRONTERA COMERCIAL',
       'CODIGO CIRCUITO O LINEA', 'CODIGO TRANSFORMADOR', 'CODIGO DANE (NIU)',
       'UBICACION', 'DIRECCION', 'CONDICIONES ESPECIALES',
       'TIPO AREA ESPECIAL', 'CODIGO AREA ESPECIAL', 'ESTRATO', 'ALTITUD',
       'LONGITUD', 'LATITUD', 'AUTOGENERADOR', 'EXPORTA ENERGIA',
       'CAPACIDAD AUTOGENERADOR (KW)', 'TIPO DE GENERACION',
       'CODIGO FRONTERA AUTOGENERACION', 'FECHA ENTRADA EN OPERACION',
       'CONTRATO DE RESPALDO', 'CAPACIDAD CONTRATO DE RESPALDO'],
      dtype='object')
Columnas en TC2: Index(['NIU', 'Tipo de Factura', 'ID Factura',
       'Fecha de Expedición de la Factura', 'Tipo de Tarifa',
       'Tipo de Lectura', 'Consumo Usuario (kWh)',
       'Valor Facturación por Consumo 

# **Empezamos con el primer paso:**
Filtrar en TC1 por ID_COMERCIALIZADOR = 23442 Y con ello verificamos el numero de clientes presentes en TC1

In [ ]:
# Filtrar por ID de comercializador 23442 y contar los NIUs en TC1
# Ajustar el nombre de la columna según la salida anterior
id_comercializador_col = 'ID COMERCIALIZADOR'  # Ajustar según sea necesario
niu_col = 'NIU'  # Ajustar según sea necesario

tc1_filtrado = tc1[tc1[id_comercializador_col] == 23442]
count_nius_tc1 = tc1_filtrado[niu_col].nunique()

print(f"Número de NIUs en TC1 después de filtrar: {count_nius_tc1}")

Número de NIUs en TC1 después de filtrar: 457597


# **Ahora Verificamos en TC2 que cumpla con el numero de clientes, para ello:**

* *Se eliminan duplicados*
* *Se valida si el número de clientes -1 un cliente, es igual al antes dado en TC1*

In [ ]:
# Paso 2: Eliminar duplicados y contar NIUs en TC2
tc2_sin_duplicados = tc2.drop_duplicates(subset='NIU')
count_nius_tc2 = tc2_sin_duplicados['NIU'].nunique()

print(f"Número de NIUs en TC2 después de eliminar duplicados: {count_nius_tc2}")

Número de NIUs en TC2 después de eliminar duplicados: 457598


In [ ]:
# Verificar que los NIUs coinciden (deben coincidir con el valor enviado por Genit menos uno)
if count_nius_tc1 == count_nius_tc2 - 1:
    print("El número de NIUs coincide con el valor esperado.")
else:
  assert count_nius_tc1 == count_nius_tc2 - 2,"El número de NIUs no coincide con el valor esperado."

El número de NIUs coincide con el valor esperado.


# **Verificamos que el cliente que es de otro mercado "898352932" corresponda con el ID de Mercado "443"**

In [ ]:
## Paso 3: Verificar el cliente en otro mercado
#clientes_otro_mercado = [898352932, 18124198] # Lista de clientes en otro mercado
#id_mercado_col = 'ID mercado'

# Filtrar el DataFrame para obtener los IDs de mercado de los clientes
#id_mercado_tc2 = tc2[tc2['NIU'].isin(clientes_otro_mercado)][id_mercado_col].iloc[0]

#if id_mercado_tc2 == 443:
#    print(f"El cliente está en el mercado con ID: {id_mercado_tc2}")
#else:
#    warnings.warn(f"Advertencia: El ID de mercado no coincide. Se encontró ID: {id_mercado_tc2}", UserWarning)


# Aquí se empieza a construir el archivo de tarifas teniendo en cuenta que este incluye:

* *NIU*
* *ESTRATO*
* *DIVIPOLA*
* *UBICACION*
* *NIVEL DE TENSION*
* *CARGA DE INVERSION*
* *ZE*

In [ ]:
# Paso 4: Crear la nueva tabla filtrada y guardarla si es necesario
columns_to_select = ['NIU', 'ESTRATO', 'CODIGO DANE (NIU)', 'UBICACION', 'NIVEL DE TENSION', 'PORCENTAJE PROPIEDAD DEL ACTIVO', 'CODIGO AREA ESPECIAL']
Tarifas = tc1_filtrado[columns_to_select]
Tarifas.columns = ['NIU', 'ESTRATO', 'DIVIPOLA', 'UBICACION', 'NIVEL DE TENSION', 'CARGA DE INVERSION', 'ZE']

In [ ]:
Tarifas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 457597 entries, 0 to 458133
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   NIU                 457597 non-null  object
 1   ESTRATO             457597 non-null  int64 
 2   DIVIPOLA            457597 non-null  int64 
 3   UBICACION           457597 non-null  int64 
 4   NIVEL DE TENSION    457597 non-null  int64 
 5   CARGA DE INVERSION  457597 non-null  int64 
 6   ZE                  457597 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 27.9+ MB


# Ahora se le tienen que hacer algunas modificaciones en este caso:

* *El estrato esta condicionado asi 7=I - 8=C - 9=O - 11=AP*
* *La ubicación esta de la siguiente forma 1=R (Rural) - 2=U (Urbano)*
* *La carga de inversión donde en 101=0*

In [ ]:
# Modificar valores en 'ESTRATO'
Tarifas['ESTRATO'] = Tarifas['ESTRATO'].replace({7: 'I', 8: 'C', 9: 'O', 11: 'AP'})

# Modificar valores en 'UBICACION'
Tarifas['UBICACION'] = Tarifas['UBICACION'].replace({1: 'R', 2: 'U'})

#Modificar valores de "CARGA DE INVERSION"
Tarifas['CARGA DE INVERSION'] = Tarifas['CARGA DE INVERSION'].replace({101:0})

<ipython-input-23-da6e03f3dd47>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tarifas['ESTRATO'] = Tarifas['ESTRATO'].replace({7: 'I', 8: 'C', 9: 'O', 11: 'AP'})
<ipython-input-23-da6e03f3dd47>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tarifas['UBICACION'] = Tarifas['UBICACION'].replace({1: 'R', 2: 'U'})
<ipython-input-23-da6e03f3dd47>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [ ]:
Tarifas.head()

,NIU,ESTRATO,DIVIPOLA,UBICACION,NIVEL DE TENSION,CARGA DE INVERSION,ZE
0,225309471,I,19622000,U,1,100,0
1,225310207,I,19532000,R,1,100,0
2,225314435,O,19532000,R,1,100,0
3,225315112,C,19392000,U,1,100,0
4,225321963,I,19397000,U,1,0,0


# Ahora se traen los nombre de los municipios según correspondan al código DAVIPOLA

In [ ]:
# Realiza la combinación de los DataFrames
Tarifas1 = Tarifas.merge(davipola[['Código DIVIPOLA', 'Nombre Municipio ']],
                        left_on='DIVIPOLA', right_on='Código DIVIPOLA', how='left')

# Renombra la nueva columna con el nombre del municipio
Tarifas1 = Tarifas1.rename(columns={'Nombre Municipio ': 'Municipio'})

# Elimina la columna 'Código DIVIPOLA' si no es necesaria
Tarifas1 = Tarifas1.drop(columns=['Código DIVIPOLA'])

In [ ]:
Tarifas1.head()

,NIU,ESTRATO,DIVIPOLA,UBICACION,NIVEL DE TENSION,CARGA DE INVERSION,ZE,Municipio
0,225309471,I,19622000,U,1,100,0,ROSAS
1,225310207,I,19532000,R,1,100,0,PATIA
2,225314435,O,19532000,R,1,100,0,PATIA
3,225315112,C,19392000,U,1,100,0,LA SIERRA
4,225321963,I,19397000,U,1,0,0,LA VEGA


# Se verifica que los NIU que estan duplicados cuenten con el mismo tipo de tarifa

In [ ]:
#Verificar tipo de tarifa en los NIU repetidos
# Paso 1: Encontrar NIU duplicados
duplicated_nius = tc2[tc2.duplicated(subset='NIU', keep=False)]

# Paso 2: Verificar si esos NIU duplicados tienen el mismo tipo de tarifa
different_tarifas = duplicated_nius.groupby('NIU')['Tipo de Tarifa'].nunique()

# Paso 3: Filtrar los NIU que tienen más de un tipo de tarifa
nius_with_different_tarifas = different_tarifas[different_tarifas > 1]

# Paso 4: Imprimir los resultados
if not nius_with_different_tarifas.empty:
    print("NIU igual, pero con tipo de tarifa diferente son:")
    niu_different_tarifa_df = duplicated_nius[duplicated_nius['NIU'].isin(nius_with_different_tarifas.index)]
    print(niu_different_tarifa_df[['NIU', 'Tipo de Tarifa']])

    # Asegurarte de que la cantidad de NIUs con diferentes tarifas sea cero
    assert niu_different_tarifa_df.empty, "Error: Hay NIUs con diferentes tipos de tarifa."
else:
    print("TODO BIEN CON LOS TIPOS DE TARIFA!")


TODO BIEN CON LOS TIPOS DE TARIFA!


# Paso 2. Se traen los siguientes valores de TC2:

* *Consumo Usuario (Con tabla dinámica)*
* *Valor facturación usuario (Con tabla dinámica)*
* *Tipo de tarifas (Sin tabla dinámica)*


Como paso añadido se tiene que cambiar el tipo dew tarifa de la siguiente forma:
 * *Tarifa 2 = NR (No regulada)*
 * *Tarifa 1 = R (Regulada)*

In [ ]:
# Crear la tabla dinámica sumando los valores
pivot_table = pd.pivot_table(tc2,
                             index='NIU',
                             values=['Consumo Usuario (kWh)', 'Valor Facturación por Consumo Usuario'],
                             aggfunc='sum')  # Usamos 'sum' para agregar los valores

# Reiniciar el índice para que 'NIU' sea una columna
pivot_table.reset_index(inplace=True)

# Extraer las columnas requeridas en un nuevo DataFrame
tblDinamicaTc2 = pivot_table[['NIU', 'Consumo Usuario (kWh)', 'Valor Facturación por Consumo Usuario']]

In [ ]:
# Convertir las columnas NIU a tipo string y eliminar espacios en blanco en ambos DataFrames
Tarifas1['NIU'] = Tarifas1['NIU'].astype(str).str.strip()
tblDinamicaTc2['NIU'] = tblDinamicaTc2['NIU'].astype(str).str.strip()
tc2_sin_duplicados['NIU'] = tc2_sin_duplicados['NIU'].astype(str).str.strip()

<ipython-input-29-f1a63ac7a0aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tc2_sin_duplicados['NIU'] = tc2_sin_duplicados['NIU'].astype(str).str.strip()


In [ ]:
# Realizar la combinación para añadir 'Tipo_tarifa' basado en 'NIU'
tblDinamicaTc2 = tblDinamicaTc2.merge(tc2_sin_duplicados[['NIU', 'Tipo de Tarifa']], on='NIU', how='left')
# Realizar la combinación para añadir las columnas que faltan de tblDinamicaTc2 a Tarifas1
Tarifas2 = Tarifas1.merge(tblDinamicaTc2[['NIU', 'Consumo Usuario (kWh)', 'Valor Facturación por Consumo Usuario', 'Tipo de Tarifa']],
                          on='NIU', how='left')

In [ ]:
# Filtrar el DataFrame `tc2` por NIU
filtrado = tc2_sin_duplicados[tc2_sin_duplicados['NIU'] == '225361559']
filtrado.head()

,NIU,Tipo de Factura,ID Factura,Fecha de Expedición de la Factura,Tipo de Tarifa,Tipo de Lectura,Consumo Usuario (kWh),Valor Facturación por Consumo Usuario,Refacturación por Consumo Usuario (kwh),Valor Refacturación por Consumo Usuario,Tipo Medidor,Código Medidor
457708,225361559,5,NaN,NaT,1,1,0.0,0.0,0.0,0.0,2,50119253ITRTM


In [ ]:
# Modificar valores en 'Tipo_tarifa'
Tarifas2['Tipo de Tarifa'] = Tarifas2['Tipo de Tarifa'].replace({1: 'R', 2: 'NR'})

In [ ]:
# Mostrar el DataFrame resultante
Tarifas2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457597 entries, 0 to 457596
Data columns (total 11 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   NIU                                    457597 non-null  object 
 1   ESTRATO                                457597 non-null  object 
 2   DIVIPOLA                               457597 non-null  int64  
 3   UBICACION                              457597 non-null  object 
 4   NIVEL DE TENSION                       457597 non-null  int64  
 5   CARGA DE INVERSION                     457597 non-null  int64  
 6   ZE                                     457597 non-null  int64  
 7   Municipio                              457597 non-null  object 
 8   Consumo Usuario (kWh)                  457597 non-null  float64
 9   Valor Facturación por Consumo Usuario  457597 non-null  float64
 10  Tipo de Tarifa                         457597 non-null  

# Ahora se organizan las columnas en el orden que se debe presentar:

* *NIU*
* *ESTRATO*
* *TIPO DE TARIFA*
* *CONSUMO*
* *FACTURACION CONSUMO*
* *UBICACION*
* *DAVIPOLA*
* *MUNICIPIO*
* *NIVEL DE TENSION*
* *CARGO DE INVERSION*
* *ZE*

In [ ]:
# Reorganizar las columnas en el orden deseado
Tarifas3 = Tarifas2[['NIU', 'ESTRATO', 'Tipo de Tarifa', 'Consumo Usuario (kWh)',
                                         'Valor Facturación por Consumo Usuario', 'UBICACION',
                                         'DIVIPOLA', 'Municipio', 'NIVEL DE TENSION',
                                         'CARGA DE INVERSION', 'ZE']]

# Renombrar las columnas según los nuevos nombres proporcionados
Tarifas3 = Tarifas3.rename(columns={
    'Tipo de Tarifa': 'TIPO TARIFA',
    'Consumo Usuario (kWh)': 'CONSUMO',
    'Valor Facturación por Consumo Usuario': 'FACTURACION CONSUMO',
    'Municipio': 'MUNICIPIO',
    'DIVIPOLA': 'DAVIPOLA'
})

In [ ]:
Tarifas3.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
0,225309471,I,R,0.0,0.00,U,19622000,ROSAS,1,100,0
1,225310207,I,R,1684.0,1677500.77,R,19532000,PATIA,1,100,0
2,225314435,O,R,2080.0,2071972.45,R,19532000,PATIA,1,100,0
3,225315112,C,R,79.0,78695.11,U,19392000,LA SIERRA,1,100,0
4,225321963,I,R,0.0,0.00,U,19397000,LA VEGA,1,0,0


# Añadir el Cliente de otro mercado

In [ ]:
# Filtrar el DataFrame `tc2` por NIU
niu_filtrado = tc2[(tc2['NIU'] == 898352932) | (tc2['NIU'] == 18124198)]

# Extraer los valores de Consumo Usuario y Valor Facturación por Consumo Usuario ($)
consumo_usuario = niu_filtrado['Consumo Usuario (kWh)'].values[0]
valor_facturacion = niu_filtrado['Valor Facturación por Consumo Usuario'].values[0]

# Crear un nuevo DataFrame con la fila que quieres añadir
nueva_fila = pd.DataFrame({
    'NIU': [898352932],
    'ESTRATO': ['I'],
    'TIPO TARIFA': ['NR'],
    'CONSUMO': [consumo_usuario],
    'FACTURACION CONSUMO': [valor_facturacion],
    'UBICACION': ['U'],
    'DAVIPOLA': [13001000],
    'MUNICIPIO': ['CARTAGENA'],
    'NIVEL DE TENSION': [2],
    'CARGA DE INVERSION': [0],
    'ZE': [0]
})

In [ ]:
# Añadir esta fila al DataFrame Tarifas3
Tarifas3 = pd.concat([Tarifas3, nueva_fila], ignore_index=True)

In [ ]:
Tarifas3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457598 entries, 0 to 457597
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   NIU                  457598 non-null  object 
 1   ESTRATO              457598 non-null  object 
 2   TIPO TARIFA          457598 non-null  object 
 3   CONSUMO              457598 non-null  float64
 4   FACTURACION CONSUMO  457598 non-null  float64
 5   UBICACION            457598 non-null  object 
 6   DAVIPOLA             457598 non-null  int64  
 7   MUNICIPIO            457598 non-null  object 
 8   NIVEL DE TENSION     457598 non-null  int64  
 9   CARGA DE INVERSION   457598 non-null  int64  
 10  ZE                   457598 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 38.4+ MB


# Se eliminan los NIU = CALP

In [ ]:
# Convertir la columna NIU a tipo string y reemplazar NaN con una cadena vacía
Tarifas3['NIU'] = Tarifas3['NIU'].astype(str).fillna('')

# Filtrar el DataFrame para eliminar filas donde NIU contiene 'CAL'
Tarifas_sin_cal = Tarifas3[~Tarifas3['NIU'].str.contains('CAL')]

In [ ]:
Tarifas_sin_cal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 454730 entries, 0 to 457597
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   NIU                  454730 non-null  object 
 1   ESTRATO              454730 non-null  object 
 2   TIPO TARIFA          454730 non-null  object 
 3   CONSUMO              454730 non-null  float64
 4   FACTURACION CONSUMO  454730 non-null  float64
 5   UBICACION            454730 non-null  object 
 6   DAVIPOLA             454730 non-null  int64  
 7   MUNICIPIO            454730 non-null  object 
 8   NIVEL DE TENSION     454730 non-null  int64  
 9   CARGA DE INVERSION   454730 non-null  int64  
 10  ZE                   454730 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 41.6+ MB


# Se trae del archivo AP entregado por Hector al archivo de tarifas:

* *TIPO DE TARIFA*
* *CONSUMO USUARIO*
* *FACTURACION CONSUMO USUARIO*

In [ ]:
#Convertir los elementos de la columna productos a str
archivo_ap['producto'] = archivo_ap['producto'].astype(str).str.strip()

#Validamos que el archivo AP no contenga productos vacíos
if archivo_ap['producto'].eq('').any():
    raise ValueError("El archivo AP contiene productos vacíos. Por favor, corrige los datos.")
else:
    print("Validación exitosa: No hay productos vacíos en el archivo AP.")

Validación exitosa: No hay productos vacíos en el archivo AP.


In [ ]:
# Modificar valores en 'Tipo_tarifa'
archivo_ap['tipo de tarifa'] = archivo_ap['tipo de tarifa'].replace({1: 'R', 2: 'NR'})

# Modificar valores en 'ESTRATO'
archivo_ap['estrato'] = archivo_ap['estrato'].replace({11: 'AP'})

# Filtrar archivo_ap por estrato='AP'
archivo_ap_filtrado = archivo_ap[archivo_ap['estrato'] == 'AP']

In [ ]:
# Hacer un merge entre Tarifas_sin_cal y archivo_ap_filtrado basándose en NIU y producto
Tarifas4 = Tarifas_sin_cal.merge(
    archivo_ap_filtrado[['producto', 'Suma de consumo', 'Suma de facturacion consumo', 'tipo de tarifa']],
    left_on='NIU',    right_on='producto',
    how='left'
)

In [ ]:
# Filtrar archivo_ap por estrato='AP'
TarifasFiltrado = Tarifas4[Tarifas4['NIU'] == '225309471']
TarifasFiltrado.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE,producto,Suma de consumo,Suma de facturacion consumo,tipo de tarifa
0,225309471,I,R,0.0,0.0,U,19622000,ROSAS,1,100,0,NaN,NaN,NaN,NaN


In [ ]:
# Actualizar las columnas CONSUMO, FACTURACION CONSUMO y TIPO TARIFA solo si los valores son mayores a cero
Tarifas4.loc[Tarifas4['Suma de consumo'] > 0, 'CONSUMO'] = Tarifas4['Suma de consumo']
Tarifas4.loc[(Tarifas4['Suma de facturacion consumo'].notna()) & (Tarifas4['Suma de facturacion consumo'] != 0), 'FACTURACION CONSUMO'] = Tarifas4['Suma de facturacion consumo']
Tarifas4.loc[Tarifas4['tipo de tarifa'].notna(), 'TIPO TARIFA'] = Tarifas4['tipo de tarifa']

In [ ]:
# Filtrar archivo_ap por estrato='AP'
TarifasFiltrado = Tarifas4[Tarifas4['NIU'] == '225309471']
TarifasFiltrado.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE,producto,Suma de consumo,Suma de facturacion consumo,tipo de tarifa
0,225309471,I,R,0.0,0.0,U,19622000,ROSAS,1,100,0,NaN,NaN,NaN,NaN


In [ ]:
# Eliminar las columnas adicionales si no son necesarias
Tarifas4 = Tarifas4.drop(columns=['producto', 'Suma de consumo', 'Suma de facturacion consumo', 'tipo de tarifa'])

# Verificar el resultado
Tarifas4.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
0,225309471,I,R,0.0,0.00,U,19622000,ROSAS,1,100,0
1,225310207,I,R,1684.0,1677500.77,R,19532000,PATIA,1,100,0
2,225314435,O,R,2080.0,2071972.45,R,19532000,PATIA,1,100,0
3,225315112,C,R,79.0,78695.11,U,19392000,LA SIERRA,1,100,0
4,225321963,I,R,0.0,0.00,U,19397000,LA VEGA,1,0,0


In [ ]:
import numpy as np

# Redondeo personalizado
Tarifas4['CONSUMO'] = np.floor(Tarifas4['CONSUMO'] + 0.5).astype(int)
Tarifas4['FACTURACION CONSUMO'] = np.floor(Tarifas4['FACTURACION CONSUMO'] + 0.5).astype(int)

print(Tarifas4.head())

         NIU ESTRATO TIPO TARIFA  CONSUMO  FACTURACION CONSUMO UBICACION  \
0  225309471       I           R        0                    0         U   
1  225310207       I           R     1684              1677501         R   
2  225314435       O           R     2080              2071972         R   
3  225315112       C           R       79                78695         U   
4  225321963       I           R        0                    0         U   

   DAVIPOLA  MUNICIPIO  NIVEL DE TENSION  CARGA DE INVERSION  ZE  
0  19622000      ROSAS                 1                 100   0  
1  19532000      PATIA                 1                 100   0  
2  19532000      PATIA                 1                 100   0  
3  19392000  LA SIERRA                 1                 100   0  
4  19397000    LA VEGA                 1                   0   0  


In [ ]:
TarifasFiltrado = Tarifas4[Tarifas4['NIU'] == '225663404']
TarifasFiltrado.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
332760,225663404,O,R,1720,1691755,R,19130000,CAJIBIO,1,100,5101


In [ ]:
TarifasFiltrado = Tarifas4[Tarifas4['ESTRATO'] == 'AP']
TarifasFiltrado.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
273,225963805,AP,R,0,0,R,19573000,PUERTO TEJADA,2,0,0
52671,528158909,AP,R,0,0,R,19256000,EL TAMBO,2,0,5102
52672,528161309,AP,R,0,0,R,19785000,SUCRE,2,0,4113
52675,528164740,AP,R,169652,142214186,U,19548000,PIENDAMO,2,0,0
52677,528169655,AP,R,0,0,R,19397000,LA VEGA,2,0,4107


# Validación de archivo tarifas

In [ ]:
# Validación de valores vacíos en la columna NIU
if Tarifas4['NIU'].eq('').any():
    raise ValueError("Error: La columna NIU tiene valores vacíos. Revisar los archivos TC1 y TC2.")
else:
    print("Validación exitosa: La columna NIU no tiene valores vacíos.")

# Validación de valores negativos en la columna CONSUMO
if (Tarifas4['CONSUMO'] < 0).any():
    raise ValueError("Error: La columna CONSUMO tiene valores negativos. Verifica los datos.")
else:
    print("Validación exitosa: La columna CONSUMO no tiene valores negativos.")

# Validación de valores negativos en la columna FACTURACION CONSUMO
if (Tarifas4['FACTURACION CONSUMO'] < 0).any():
    raise ValueError("Error: La columna FACTURACION CONSUMO tiene valores negativos. Verifica los datos.")
else:
    print("Validación exitosa: La columna FACTURACION CONSUMO no tiene valores negativos.")

# Validación de la regla: Si CONSUMO es 0, FACTURACION CONSUMO también debe ser 0
if ((Tarifas4['CONSUMO'] == 0) & (Tarifas4['FACTURACION CONSUMO'] != 0)).any():
    raise ValueError("Error: Si CONSUMO es 0, FACTURACION CONSUMO también debe ser 0. Hay inconsistencias en los datos.")
else:
    print("Validación exitosa: No hay inconsistencias entre CONSUMO y FACTURACION CONSUMO.")

# Validación de valores nulos en todo el DataFrame
if Tarifas4.isnull().any().any():
    raise ValueError("Error: El DataFrame contiene valores nulos. Verifica las columnas y corrige los datos.")
else:
    print("Validación exitosa: El DataFrame no tiene valores nulos.")

Validación exitosa: La columna NIU no tiene valores vacíos.
Validación exitosa: La columna CONSUMO no tiene valores negativos.
Validación exitosa: La columna FACTURACION CONSUMO no tiene valores negativos.
Validación exitosa: No hay inconsistencias entre CONSUMO y FACTURACION CONSUMO.
Validación exitosa: El DataFrame no tiene valores nulos.


# Creo el Archivo Tarifas, para que se pueda descargar en la parte inferior izquierda, donde aparecera una vez se ejecute el Script.

In [ ]:
Tarifas4.to_csv('Tarifas.csv', index=False)

In [ ]:
Tarifas4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454730 entries, 0 to 454729
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   NIU                  454730 non-null  object
 1   ESTRATO              454730 non-null  object
 2   TIPO TARIFA          454730 non-null  object
 3   CONSUMO              454730 non-null  int64 
 4   FACTURACION CONSUMO  454730 non-null  int64 
 5   UBICACION            454730 non-null  object
 6   DAVIPOLA             454730 non-null  int64 
 7   MUNICIPIO            454730 non-null  object
 8   NIVEL DE TENSION     454730 non-null  int64 
 9   CARGA DE INVERSION   454730 non-null  int64 
 10  ZE                   454730 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 38.2+ MB


#**Creación de informe DANE**

In [ ]:
# Filtrar DaNE por Ubicacion='U' y Municipio='Popayán'
informeDane = Tarifas4[(Tarifas4['UBICACION'] == 'U') & (Tarifas4['MUNICIPIO'] == 'POPAYAN')]
informeDane.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
27,225389653,O,R,5037,5017560,U,19001000,POPAYAN,1,100,0
30,225394617,C,R,1020,1003250,U,19001000,POPAYAN,1,100,0
31,225398745,C,R,11396,10452417,U,19001000,POPAYAN,1,0,0
38,225407674,O,R,4736,4001871,U,19001000,POPAYAN,2,0,0
39,225409238,2,R,180,177044,U,19001000,POPAYAN,1,100,0


In [ ]:
informeDane.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110407 entries, 27 to 402060
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   NIU                  110407 non-null  object
 1   ESTRATO              110407 non-null  object
 2   TIPO TARIFA          110407 non-null  object
 3   CONSUMO              110407 non-null  int64 
 4   FACTURACION CONSUMO  110407 non-null  int64 
 5   UBICACION            110407 non-null  object
 6   DAVIPOLA             110407 non-null  int64 
 7   MUNICIPIO            110407 non-null  object
 8   NIVEL DE TENSION     110407 non-null  int64 
 9   CARGA DE INVERSION   110407 non-null  int64 
 10  ZE                   110407 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 10.1+ MB


In [ ]:
# Crear la tabla dinámica
pivot_table = informeDane.pivot_table(
    index='ESTRATO',  # Agrupar por la columna 'ESTRATO'
    values=['NIU', 'CONSUMO', 'FACTURACION CONSUMO'],  # Columnas a agregar
    aggfunc={'NIU': 'count', 'CONSUMO': 'sum', 'FACTURACION CONSUMO': 'sum'}  # Funciones de agregación
)

# Renombrar las columnas para mayor claridad
pivot_table.rename(columns={'NIU': 'CONTEO_NIU', 'CONSUMO': 'SUMA_CONSUMO', 'FACTURACION CONSUMO': 'SUMA_FACTURACION'}, inplace=True)

# Crear un nuevo DataFrame con el resultado
informeDaneVf = pivot_table.reset_index()

In [ ]:
# Mostrar el nuevo DataFrame
informeDaneVf.head(11)

,ESTRATO,SUMA_CONSUMO,SUMA_FACTURACION,CONTEO_NIU
0,1,1825129,1794349835,23815
1,2,3056768,2988848665,33191
2,3,2243047,2232282900,22029
3,4,1742084,1704627762,15983
4,5,506231,493553721,4227
5,6,108487,105765867,673
6,AP,64419,65017763,854
7,C,3713731,3457382584,8225
8,I,627474,559140952,458
9,O,902173,790209190,952


# Se guarda el archivo de tarifas, mirar al lado izquierdo (Informe_DANE.csv)

In [ ]:
informeDaneVf.to_csv('Informe_DANE.csv', index=False)

# Validación Bitacora

In [ ]:
bitacora = pd.read_excel('Bitacora.xlsx')

In [ ]:
bitacora.head(2)

,Código SIC,Producto,Tipo Frontera,Nombre,Fecha inicio,Fecha Fin,Ciudad,Departamento,Operador de Red,Agente Representante,...,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00
0,Frt00485,898157867,Tipo No Regulado,CORPORACION CLUB CAMPESTRE LOS ANDES,2013-11-30 00:00:00,NaN,Santander De quilichao,Cauca,COMPAÑIA ENERGETICA DE OCCIDENTE S.A.S. ESP - ...,EMPRESA MUNICIPAL DE ENERGIA ELECTRICA S.A. E....,...,101611.96,90205.31,109359.36,87798.43,87779.87,88024.96,94920.00,96256.26,94101.68,91891.69
1,Frt00519,898401902,Tipo No Regulado,FAMILIA DEL PACIFICO LTDA,2014-07-30 00:00:00,NaN,Caloto,Cauca,COMPAÑIA ENERGETICA DE OCCIDENTE S.A.S. ESP - ...,ISAGEN S.A. E.S.P. - COMERCIALIZADOR,...,1362660.92,1272669.12,1196250.70,1327176.19,1448595.53,1362180.84,1553802.16,2170710.48,1442829.50,1562628.68


In [ ]:
# Convertir la columna 'Producto' de bitácora a string para que coincida con el tipo de 'NIU' en tarifas4
bitacora['Producto'] = bitacora['Producto'].astype(str)
# Filtrar bitácora por 'Tipo Frontera' == 'Tipo No Regulado'
bitacora_filtrada = bitacora[bitacora['Tipo Frontera'] == 'Tipo No Regulado']

# Definir el nombre de la última columna en bitacora
ultima_columna_bitacora = bitacora.columns[-1]

# Realizar el merge en base a Producto y NIU
resultado = pd.merge(
    bitacora_filtrada[['Producto', ultima_columna_bitacora]],
    Tarifas4[['NIU', 'CONSUMO']],
    left_on='Producto', right_on='NIU',
    how='left'
)

In [ ]:
# Comparar las columnas con una tolerancia de ±1
resultado['Diferencia'] = abs(resultado[ultima_columna_bitacora] - resultado['CONSUMO'])
resultado['Es Diferente'] = resultado['Diferencia'] > 1

# Filtrar las filas que son diferentes
diferencias = resultado[resultado['Es Diferente']][['NIU', 'CONSUMO', ultima_columna_bitacora]]

In [ ]:
# Mostrar el DataFrame con las diferencias
print(diferencias)

          NIU   CONSUMO  2024-10-01 00:00:00
32  898156703   34452.0             35134.14
36  892349233  244170.0            257635.29
40  892348456   74035.0             75418.71
42  898113707   49126.0             49398.81
58  666080702       0.0             10472.86
95  898744084  121932.0            139718.02


In [ ]:
TarifasFiltrado = Tarifas4[Tarifas4['NIU'] == '228730870']
TarifasFiltrado.head()

,NIU,ESTRATO,TIPO TARIFA,CONSUMO,FACTURACION CONSUMO,UBICACION,DAVIPOLA,MUNICIPIO,NIVEL DE TENSION,CARGA DE INVERSION,ZE
330061,228730870,1,R,130,127865,R,19532000,PATIA,1,100,4240


# codigo realizado:
Humberto Aldemir Fajardo Castaño
https://github.com/Humberto3280